# Location Decision of a Museum Service Firm


## Import Libraries

In [3]:
import requests 
#!pip install beautifulsoup4
from bs4 import BeautifulSoup 
import pandas as pd 
import numpy as np
#!conda install -c anaconda lxml --yes
import lxml 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sn
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Scrape a list of the top US cities (by population) with at least 100,000 people

In [7]:
#sent the GET Request 
url='https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(url)
#Store the contents of the website under doc
doc = page.text

## Scraping Table data with BeautifulSoup

In [8]:
soup=BeautifulSoup(doc, 'html.parser')
table=soup.find('table', {'class': 'wikitable sortable'})

## Read and convert table into a pandas DataFrame

In [9]:
df1=pd.read_html(str(table), header=0)
df1
df=pd.DataFrame(df1[0])
df[0:5]

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York[d],New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston[3],Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


In [10]:
df['City']=df['City'].str.split('\[', expand=True)
df.head()

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York,New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston,Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


In [11]:
df.rename(columns={'State[c]': 'State'}, inplace=True)

In [12]:
df.head()

,2019rank,City,State,2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York,New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston,Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


In [13]:
df_cities=df[['2019rank', 'City', 'State', '2019estimate']]
df_cities.head()

,2019rank,City,State,2019estimate
0,1,New York,New York,8336817
1,2,Los Angeles,California,3979576
2,3,Chicago,Illinois,2693976
3,4,Houston,Texas,2320268
4,5,Phoenix,Arizona,1680992


## Find the Longitude and Latitude Coordinates for each City

In [16]:
geolocator = Nominatim(user_agent="ny_explorer")
results=[('City',[]), ('State', []), ('Latitude', []), ('Longitude',[])]
for i in range(0,len(df_cities.index)):
    state=df_cities.iloc[i,2]
    city=df_cities.iloc[i,1]
    address = '{},{}'.format(city, state)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    results[0][1].append(city)
    results[1][1].append(state)
    results[2][1].append(latitude)
    results[3][1].append(longitude)


In [19]:
dict={title:column for (title, column) in results} # convert to a dictionary
df_cord=pd.DataFrame(dict)                            # create a pandas dataFrame from the dictionary

In [22]:
df_cord.head()

,City,State,Latitude,Longitude
0,New York,New York,40.712728,-74.006015
1,Los Angeles,California,34.053691,-118.242767
2,Chicago,Illinois,41.875562,-87.624421
3,Houston,Texas,29.758938,-95.367697
4,Phoenix,Arizona,33.448437,-112.074142


In [26]:
df_cord.to_csv('df_cord.csv')

In [2]:
df_cord=pd.read_csv('df_cord.csv', index_col=0)

In [3]:
df_cord.shape

(314, 4)

###### Define Foursquare Credentials and version

In [6]:
CLIENT_ID = 'D0HIFCALFN4KTTJCPZEXUFFPYRTA2LOZIXTGWBXRWF4TEX1I' # your Foursquare ID
CLIENT_SECRET = 'NIMTP13QNEAOHNFYQCSG3AB3OKGLREJH4AVJOFK4YAOSCM1R' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D0HIFCALFN4KTTJCPZEXUFFPYRTA2LOZIXTGWBXRWF4TEX1I
CLIENT_SECRET:NIMTP13QNEAOHNFYQCSG3AB3OKGLREJH4AVJOFK4YAOSCM1R


## Define the Search Querry

In [7]:
search_query='Museum'

## Search for the Top Museums in each of the cities in the list

In [47]:
venues_list=[]
for i in range(0,len(df_cities.index)):
    state=df_cities.iloc[i,2]
    city=df_cities.iloc[i,1]
    address = '{},{}'.format(city, state)
    url = 'https://api.foursquare.com/v2/venues/explore?near={},{}&client_id={}&client_secret={}&v={}&query={}'.format(city, state, CLIENT_ID, CLIENT_SECRET, VERSION, search_query)
    # make the GET request
    res = requests.get(url).json()
    if res['meta']['code']==200:
        results=res['response']['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(city, state, v['venue']['id'], v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name'],v['venue']['categories'][0]['id']) for v in results])

city_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
city_venues.columns = ['City','State','venue_id','Venue', 'VenueLatitude', 'VenueLongitude', 'Category', 'Category_id']
    

In [60]:
city_venues.head()

,City,State,venue_id,Venue,VenueLatitude,VenueLongitude,Category,Category_id
0,New York,New York,427c0500f964a52097211fe3,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum,4bf58dd8d48988d18f941735
1,New York,New York,4a746fb2f964a52025de1fe3,The Jewish Museum,40.785276,-73.957411,Museum,4bf58dd8d48988d181941735
2,New York,New York,4f2715dce4b063d40326c3cd,MoMA: Architecture and Design,40.761547,-73.976732,Art Museum,4bf58dd8d48988d18f941735
3,New York,New York,421a7600f964a5209d1f1fe3,Whitney Museum of American Art,40.739506,-74.008861,Art Museum,4bf58dd8d48988d18f941735
4,New York,New York,49dd262cf964a520eb5f1fe3,Japan Society,40.752287,-73.968431,Museum,4bf58dd8d48988d181941735


In [40]:
city_venues.to_csv('df_venues.csv')

In [16]:
df_venues=pd.read_csv('df_venues.csv', index_col=0)
df_venues.head()

,City,State,venue_id,Venue,VenueLatitude,VenueLongitude,Category,Category_id
0,New York,New York,427c0500f964a52097211fe3,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum,4bf58dd8d48988d18f941735
1,New York,New York,4a746fb2f964a52025de1fe3,The Jewish Museum,40.785276,-73.957411,Museum,4bf58dd8d48988d181941735
2,New York,New York,4f2715dce4b063d40326c3cd,MoMA: Architecture and Design,40.761547,-73.976732,Art Museum,4bf58dd8d48988d18f941735
3,New York,New York,421a7600f964a5209d1f1fe3,Whitney Museum of American Art,40.739506,-74.008861,Art Museum,4bf58dd8d48988d18f941735
4,New York,New York,49dd262cf964a520eb5f1fe3,Japan Society,40.752287,-73.968431,Museum,4bf58dd8d48988d181941735


## Find venue specific information such as rating, number of ratings(ratingCount), number of tips(tipCount) and number of likes(likesCount) for each Museum

In [11]:
venues_info=[]
for i in range(0,len(df_venues.index)):
    venue_id=df_venues.iloc[i,2]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    # make the GET request
    result = requests.get(url).json()
    if result['meta']['code']==200:
        try:
            rating=result['response']['venue']['rating']
        except:
            rating=np.nan
        try:
            ratingCount=result['response']['venue']['ratingSignals']
        except:
            ratingCount=np.nan
        try:
            tipCount=result['response']['venue']['stats']['tipCount']
        except:
            tipCount=np.nan
        try:
            likesCount=result['response']['venue']['likes']['count']
        except:
            likesCount=np.nan
    else:
        rating='Not Found'
        ratingCount='Not Found'
        tipCount='Not Found'
        likesCount='Not Found'
    venues_info.append([venue_id, rating, ratingCount, tipCount, likesCount])

venues_data = pd.DataFrame(venues_info, columns=['venue_id', 'rating', 'ratingCount', 'tipCount', 'likesCount' ])
    

In [26]:
venues_data.head()


,venue_id,rating,ratingCount,tipCount,likesCount
0,427c0500f964a52097211fe3,9.6,16054,1935,14199
1,4a746fb2f964a52025de1fe3,9.4,886,72,791
2,4f2715dce4b063d40326c3cd,9.2,281,24,252
3,421a7600f964a5209d1f1fe3,9.2,5053,425,4319
4,49dd262cf964a520eb5f1fe3,9.2,276,47,226


## Merge the Cites data with the Musuem Info

In [21]:
cities_venues.head()

,venue_id,City,State,Venue,VenueLatitude,VenueLongitude,Category,Category_id,rating,ratingCount,tipCount,likesCount
0,40943a00f964a520e5f21ee3,San Francisco,California,San Francisco Museum of Modern Art,37.785894,-122.400897,Art Museum,4bf58dd8d48988d18f941735,9.3,2288.0,238,1961.0
1,40a6af00f964a52027f31ee3,Los Angeles,California,Los Angeles County Museum of Art (LACMA),34.063242,-118.359563,Art Museum,4bf58dd8d48988d18f941735,9.4,3342.0,442,2877.0
2,41424000f964a5208d1c1fe3,Philadelphia,Pennsylvania,Philadelphia Museum of Art,39.965806,-75.181225,Art Museum,4bf58dd8d48988d18f941735,9.4,1989.0,306,1678.0
3,41706480f964a520a51d1fe3,New York,New York,Solomon R Guggenheim Museum,40.783051,-73.959265,Art Museum,4bf58dd8d48988d18f941735,8.7,5154.0,568,4121.0
4,421a7600f964a5209d1f1fe3,New York,New York,Whitney Museum of American Art,40.739506,-74.008861,Art Museum,4bf58dd8d48988d18f941735,9.2,5053.0,425,4319.0


In [20]:
cities_venues.shape

(4126, 12)

In [22]:
len(cities_venues['venue_id'].unique())

3626

In [26]:
cities_venues=cities_venues.drop_duplicates()

In [27]:
cities_venues.shape

(3626, 12)

In [28]:
cities_venues.to_csv('cities_venues.csv')

In [14]:
df_venues=pd.read_csv('df_venues.csv', index_col=0)

In [15]:
df_venues=pd.read_csv('df_venues.csv', index_col=0)

In [16]:
df_venues.sort_values(by='venue_id').head()

,City,State,venue_id,Venue,VenueLatitude,VenueLongitude,Category,Category_id
398,San Francisco,California,40943a00f964a520e5f21ee3,San Francisco Museum of Modern Art,37.785894,-122.400897,Art Museum,4bf58dd8d48988d18f941735
4982,Richmond,California,40943a00f964a520e5f21ee3,San Francisco Museum of Modern Art,37.785894,-122.400897,Art Museum,4bf58dd8d48988d18f941735
31,Los Angeles,California,40a6af00f964a52027f31ee3,Los Angeles County Museum of Art (LACMA),34.063242,-118.359563,Art Museum,4bf58dd8d48988d18f941735
2030,Hialeah,Florida,40e0b100f964a5207c051fe3,Vizcaya Museum and Gardens,25.745229,-80.211213,Museum,4bf58dd8d48988d181941735
1021,Miami,Florida,40e0b100f964a5207c051fe3,Vizcaya Museum and Gardens,25.745229,-80.211213,Museum,4bf58dd8d48988d181941735


In [17]:
df_venues=df_venues.reset_index(drop=True)

In [18]:
df_venues=df_venues[['City', 'State', 'venue_id']]

In [ ]:
df=pd.merge(df_venues, cities_venues, on=['venue_id'])

In [69]:
df=df.drop(['City_y', 'State_y'], axis=1)

In [84]:
df.to_csv('df.csv')

In [22]:
df=pd.read_csv('df.csv', index_col=0)

In [23]:
df=df.dropna(subset=['rating'])

In [24]:
df.head()

,City,State,venue_id,Venue,VenueLatitude,VenueLongitude,Category,Category_id,rating,ratingCount,tipCount,likesCount
0,New York,New York,427c0500f964a52097211fe3,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum,4bf58dd8d48988d18f941735,9.6,16054.0,1935,14199.0
1,New York,New York,4a746fb2f964a52025de1fe3,The Jewish Museum,40.785276,-73.957411,Museum,4bf58dd8d48988d181941735,9.4,886.0,72,791.0
2,New York,New York,4f2715dce4b063d40326c3cd,MoMA: Architecture and Design,40.761547,-73.976732,Art Museum,4bf58dd8d48988d18f941735,9.2,281.0,24,252.0
3,New York,New York,421a7600f964a5209d1f1fe3,Whitney Museum of American Art,40.739506,-74.008861,Art Museum,4bf58dd8d48988d18f941735,9.2,5053.0,425,4319.0
4,Newark,New Jersey,421a7600f964a5209d1f1fe3,Whitney Museum of American Art,40.739506,-74.008861,Art Museum,4bf58dd8d48988d18f941735,9.2,5053.0,425,4319.0


In [25]:
df.shape

(3463, 12)

# Count the number of Museums in Each City

In [26]:
df['N']=df.groupby('City')['rating'].transform('count')

In [27]:
df.head()

,City,State,venue_id,Venue,VenueLatitude,VenueLongitude,Category,Category_id,rating,ratingCount,tipCount,likesCount,N
0,New York,New York,427c0500f964a52097211fe3,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum,4bf58dd8d48988d18f941735,9.6,16054.0,1935,14199.0,30
1,New York,New York,4a746fb2f964a52025de1fe3,The Jewish Museum,40.785276,-73.957411,Museum,4bf58dd8d48988d181941735,9.4,886.0,72,791.0,30
2,New York,New York,4f2715dce4b063d40326c3cd,MoMA: Architecture and Design,40.761547,-73.976732,Art Museum,4bf58dd8d48988d18f941735,9.2,281.0,24,252.0,30
3,New York,New York,421a7600f964a5209d1f1fe3,Whitney Museum of American Art,40.739506,-74.008861,Art Museum,4bf58dd8d48988d18f941735,9.2,5053.0,425,4319.0,30
4,Newark,New Jersey,421a7600f964a5209d1f1fe3,Whitney Museum of American Art,40.739506,-74.008861,Art Museum,4bf58dd8d48988d18f941735,9.2,5053.0,425,4319.0,30


## Average out the ratings, ratingCounts, tipCount, likeCount for each City

In [35]:
df_grouped = df.groupby(['City', 'State']).mean().reset_index()

In [36]:
df_grouped=df_grouped[['City', 'State', 'rating', 'ratingCount', 'tipCount', 'likesCount', 'N']]
df_grouped.head()

,City,State,rating,ratingCount,tipCount,likesCount,N
0,Abilene,Texas,7.333333,18.666667,6.333333,10.666667,3.0
1,Akron,Ohio,8.800000,113.500000,24.500000,88.500000,2.0
2,Albuquerque,New Mexico,7.962500,76.625000,13.875000,57.625000,8.0
3,Alexandria,Virginia,7.172727,19.000000,3.727273,13.636364,11.0
4,Allen,Texas,7.485714,20.428571,3.428571,15.285714,7.0


In [37]:
df_grouped.sort_values('N').head()

,City,State,rating,ratingCount,tipCount,likesCount,N
76,El Cajon,California,8.9,96.0,18.0,74.0,1.0
234,Salinas,California,7.4,61.0,10.0,42.0,1.0
152,Lowell,Massachusetts,6.7,31.0,6.0,19.0,1.0
197,Palm Bay,Florida,7.5,14.0,2.0,10.0,1.0
198,Palmdale,California,7.8,10.0,2.0,8.0,1.0


In [38]:
round(df_grouped.describe(), 2)


,rating,ratingCount,tipCount,likesCount,N
count,303.00,303.00,303.00,303.00,303.00
mean,7.75,98.65,16.79,77.77,12.56
std,0.44,161.92,20.04,137.19,10.92
min,6.42,9.33,1.00,5.00,1.00
25%,7.48,31.65,7.00,21.33,4.00
50%,7.79,60.72,11.71,45.20,9.00
75%,8.01,112.85,21.91,88.75,19.00
max,8.98,2265.83,265.60,1932.60,59.00


## Cluster the Cities based on the Cities Average of rating, ratingCount, tipCount, likesCount

In [39]:
X=df_grouped[['rating', 'ratingCount', 'tipCount', 'likesCount', 'N']]

In [40]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [41]:
df_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [42]:
df_grouped.sort_values('Cluster Labels')[0:5]

,Cluster Labels,City,State,rating,ratingCount,tipCount,likesCount,N
0,0,Abilene,Texas,7.333333,18.666667,6.333333,10.666667,3.0
140,0,Lancaster,California,7.800000,10.000000,2.000000,8.000000,1.0
141,0,Lansing,Michigan,7.675000,34.000000,8.000000,25.250000,4.0
142,0,Las Cruces,New Mexico,7.866667,20.000000,3.333333,15.000000,3.0
259,0,Springfield,Missouri,7.450000,34.250000,7.750000,22.750000,14.0


## Add in the Latititude and Longitude Coordinates of each City 

In [43]:
df_cord=pd.read_csv('df_cord.csv', index_col=0)

In [44]:
df_clustered=pd.merge(df_grouped, df_cord, on=['City', 'State'])

In [45]:
df_clustered.head()

,Cluster Labels,City,State,rating,ratingCount,tipCount,likesCount,N,Latitude,Longitude
0,0,Abilene,Texas,7.333333,18.666667,6.333333,10.666667,3.0,32.446450,-99.747591
1,4,Akron,Ohio,8.800000,113.500000,24.500000,88.500000,2.0,41.083064,-81.518485
2,0,Albuquerque,New Mexico,7.962500,76.625000,13.875000,57.625000,8.0,35.084103,-106.650985
3,0,Alexandria,Virginia,7.172727,19.000000,3.727273,13.636364,11.0,38.805110,-77.047023
4,0,Allen,Texas,7.485714,20.428571,3.428571,15.285714,7.0,33.103174,-96.670550


In [46]:
df_clusteredgrouped = df_grouped.groupby(['Cluster Labels']).mean().reset_index()
df_clusteredgrouped = df_clusteredgrouped[['Cluster Labels','rating', 'ratingCount', 'tipCount', 'likesCount', 'N']]
df_clusteredgrouped.sort_values(by=['ratingCount'], ascending=False)

,Cluster Labels,rating,ratingCount,tipCount,likesCount,N
1,1,8.976667,2265.833333,265.600000,1932.600000,30.000000
2,2,8.685402,588.017570,77.050411,491.481445,32.571429
3,3,8.165059,235.738679,35.522283,190.433557,23.071429
4,4,7.971613,115.922321,21.213003,90.657796,18.166667
0,0,7.549628,39.191582,8.224405,28.659822,7.524590


## Visualize the resulting clusters on a map

In [47]:
df_top=df_grouped

In [48]:
# create map
longitude=-95.712891
latitude=37.090240
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, city, cluster in zip(df_clustered['Latitude'], df_clustered['Longitude'], df_clustered['City'], df_clustered['Cluster Labels']):
    label = folium.Popup(str(city) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Which Clusters have the best Museums (Musuems with high rating, high number of rating counts, tipCounts, and likesCount)

In [49]:
df_clusteredgrouped = df_grouped.groupby(['Cluster Labels']).mean().reset_index()
df_clusteredgrouped.sort_values(by=['ratingCount'], ascending=False)

,Cluster Labels,rating,ratingCount,tipCount,likesCount,N
1,1,8.976667,2265.833333,265.600000,1932.600000,30.000000
2,2,8.685402,588.017570,77.050411,491.481445,32.571429
3,3,8.165059,235.738679,35.522283,190.433557,23.071429
4,4,7.971613,115.922321,21.213003,90.657796,18.166667
0,0,7.549628,39.191582,8.224405,28.659822,7.524590


In [50]:
print('The top Musuems are in Clusters 1 and 2, followed by Cluster 3')

The top Musuems are in Clusters 1 and 2, followed by Cluster 3


## Which Cities are in Clusters 1 and 2 (the top Clusters)

In [51]:
df_top=df_clustered[(df_clustered['Cluster Labels']==1) | (df_clustered['Cluster Labels']==2)]

In [52]:
df_top=df_top[['Cluster Labels', 'City', 'State', 'rating', 'ratingCount', 'tipCount', 'likesCount', 'N']]

In [53]:
df_top.sort_values(by=['Cluster Labels'])

,Cluster Labels,City,State,rating,ratingCount,tipCount,likesCount,N
178,1,New York,New York,8.976667,2265.833333,265.600000,1932.600000,30.0
47,2,Chicago,Illinois,8.620000,542.066667,78.166667,446.966667,30.0
65,2,Daly City,California,8.434483,441.689655,58.586207,368.103448,29.0
150,2,Los Angeles,California,8.846667,668.200000,88.266667,567.366667,30.0
179,2,Newark,New Jersey,8.576667,498.300000,66.366667,409.300000,30.0
223,2,Richmond,California,8.660000,438.766667,56.033333,366.600000,49.0
240,2,San Francisco,California,8.746667,640.133333,82.933333,535.666667,30.0
292,2,Washington,District of Columbia,8.913333,886.966667,109.000000,746.366667,30.0


## Which Cities are in the top 10 of  Cluster 3 (the medium ranked Museum cities)

### Rank by ratingCount

In [54]:
df_middle=df_clustered[df_clustered['Cluster Labels']==3]
df_middle.sort_values(by=['ratingCount'], ascending=False)[0:10]

,Cluster Labels,City,State,rating,ratingCount,tipCount,likesCount,N,Latitude,Longitude
128,3,Jersey City,New Jersey,7.600000,384.166667,47.500000,306.833333,6.0,40.728158,-74.077642
103,3,Glendale,California,8.376667,341.166667,44.000000,287.733333,59.0,34.146942,-118.247847
250,3,Seattle,Washington,8.314815,314.407407,49.222222,250.481481,27.0,47.603832,-122.330062
184,3,North Las Vegas,Nevada,7.900000,304.400000,57.800000,234.200000,5.0,36.200584,-115.121584
28,3,Boston,Massachusetts,8.336667,292.900000,45.233333,236.666667,30.0,42.360253,-71.058291
23,3,Bellevue,Washington,8.286667,288.166667,45.466667,229.233333,30.0,47.614422,-122.192337
206,3,Philadelphia,Pennsylvania,8.496667,274.033333,44.266667,221.166667,30.0,39.952724,-75.163526
217,3,Raleigh,North Carolina,8.614286,253.571429,33.285714,214.000000,7.0,35.780398,-78.639099
14,3,Atlanta,Georgia,8.460000,247.100000,46.400000,189.833333,30.0,33.749099,-84.390185
243,3,Sandy Springs,Georgia,8.370000,240.033333,45.366667,184.200000,30.0,33.924269,-84.378538


In [30]:
df_middle.shape

(28, 11)

#### Rank by ranksize, a product of rating, ratingCount, tipCount,likesCount, and N

In [55]:
df_middle['ranksize']=df_middle['rating']*df_middle['ratingCount']*df_middle['tipCount']*df_middle['likesCount']*df_middle['N']

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
df_middle.sort_values(by=['ranksize'], ascending=False)[0:8]

,Cluster Labels,City,State,rating,ratingCount,tipCount,likesCount,N,Latitude,Longitude,ranksize
103,3,Glendale,California,8.376667,341.166667,44.000000,287.733333,59.0,34.146942,-118.247847,2.134680e+09
250,3,Seattle,Washington,8.314815,314.407407,49.222222,250.481481,27.0,47.603832,-122.330062,8.702539e+08
28,3,Boston,Massachusetts,8.336667,292.900000,45.233333,236.666667,30.0,42.360253,-71.058291,7.842035e+08
23,3,Bellevue,Washington,8.286667,288.166667,45.466667,229.233333,30.0,47.614422,-122.192337,7.466477e+08
206,3,Philadelphia,Pennsylvania,8.496667,274.033333,44.266667,221.166667,30.0,39.952724,-75.163526,6.838640e+08
14,3,Atlanta,Georgia,8.460000,247.100000,46.400000,189.833333,30.0,33.749099,-84.390185,5.524015e+08
243,3,Sandy Springs,Georgia,8.370000,240.033333,45.366667,184.200000,30.0,33.924269,-84.378538,5.036685e+08
200,3,Pasadena,Texas,8.203333,215.766667,30.100000,178.566667,36.0,29.691063,-95.209101,3.424870e+08


# End